
Naive Bayes Project Tutorial

This is a simple project using Naive Bayes Classifier and Scikit-learn to create a Google Play store reviews classifier (Sentiment Analysis) in Python. You will categorize user reviews as good or bad. The Naive Bayes classification technique is a simple and powerful classification task in machine learning. The Naive Bayes classification technique is a simple and powerful classification task in machine learning. In this dataset, we use the 23 most popular mobile apps, and only two columns.


In [116]:
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install nltk
!pip install pickle


[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [117]:
import pandas as pd
import numpy as np

import nltk #text processing
import re
import unicodedata
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pickle

In [118]:
#load the words to use as filter
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True


Load Data of Reviews

In [119]:
url='https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv'
df = pd.read_csv(url)

# Explore here

In [120]:
df.head()


,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [121]:
df['polarity'].value_counts()

0    584
1    307
Name: polarity, dtype: int64

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


package name, review and polarity (0 = bad, 1 = good) 

In [123]:
df.sample(10)

,package_name,review,polarity
504,com.Slack,"now perfect, with animated gif support",1
307,com.tencent.mm,"bad new update, sight by swipe in chats gone ...",0
575,jabanaki.todo.todoly,i love this guy. he keep updating this simpl...,1
281,com.tencent.mm,apps error recently.... my phone when wanted ...,0
471,com.whatsapp,doesn't work whatsapp messages weren't sendin...,0
114,com.linkedin.android,smh linkedin will not allow me to send connec...,0
209,com.supercell.clashofclans,this new update sucks..! no more dead bases....,0
52,com.twitter.android,the app made me i'm rating it because of a po...,0
831,com.hamropatro,weather didn't show please fix it,1
141,com.king.candycrushsaga,"frequently resets once again, it has reset an...",0



## Text Processing - Clean


Step 1 - change all to lower case

In [124]:
#Preprocess the data by eliminating the package name column and putting all reviews in lower case.

df.drop(['package_name'],axis=1, inplace=True)
#chante all to lower case
df['review'] = df['review'].str.lower()

Step 2 - Remove all words that not have meaning to the search - stop words

In [125]:
#see, the word in the reviews, create a list of word and frecuency
df['review'].str.split(expand = True).stack().value_counts()[:60] #(show top 60)

the       1293
to        1159
i         1084
and        853
it         778
a          619
is         555
my         442
this       421
for        410
of         390
but        348
in         340
on         325
not        318
app        317
you        315
that       260
have       252
with       220
so         203
be         194
no         186
when       182
if         180
or         169
all        163
can        158
as         148
me         145
are        139
it's       138
can't      137
update     136
new        136
like       135
fix        133
please     131
good       128
game       121
one        119
use        118
just       117
get        112
its        111
very       109
more       107
was        104
there      104
don't      103
now        102
will       102
even       101
from       101
at         100
up          99
great       99
time        93
your        90
do          89
dtype: int64

In [126]:
#remove the stopword
stop = stopwords.words('english')

def remove_stopwords(text):
  if text is not None:
    #list of the word in the text
    words = text.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop:
        words_filtered.append(word)
    result = " ".join(words_filtered) #join the word in a text with space separation
  else:
      result = None
  return result



Eliminamos los caracteres desconocidos

In [127]:
df['review']=df['review'].str.strip()
df['review']=df['review'].str.lower()

In [128]:
def normalize_str(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result=None 
    return result

In [129]:
df['review']=df['review'].apply(normalize_str)
df['review']=df['review'].str.replace('!','')
df['review']=df['review'].str.replace(',','')
df['review']=df['review'].str.replace('&','')
df['review']=df['review'].str.normalize('NFKC')
df['review']=df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)

In [130]:
df.sample(10)

,review,polarity
877,game glitches nice you have all the cheesy sim...,0
552,it used up my battery i forgot to switch back ...,0
764,thank you thank you so much.......,1
701,still the best mini i like the new look and ne...,1
567,very bad i use xperia x8 and i think this app ...,0
321,no ringing tone whenever i call someone it sho...,0
263,good... chrome is improving... but there are s...,1
423,unfollow tool does not function unfollow contr...,0
425,this has made my life hell it really sucks my ...,0
454,good its very good to use . but i found some...,1


In [131]:
df.iloc[675,]

review      loved it rzrl app rrlrrl rrsrzrlrrl r rrsrzrlr...
polarity                                                    1
Name: 675, dtype: object

In [132]:

df_interim = df.copy()


In [133]:
df_interim['review'] = df_interim['review'].apply(remove_stopwords)

In [134]:


df_interim['review'].str.split(expand=True).stack().value_counts()[:60] 



app         351
update      150
game        140
please      138
like        137
can't       137
new         137
fix         135
good        134
use         123
one         120
great       115
get         112
even        102
time         98
browser      85
really       84
back         81
still        80
would        80
love         77
make         74
5            71
work         71
also         69
see          69
using        68
phone        68
i'm          67
open         66
it.          65
want         64
google       64
much         62
every        61
version      60
option       59
u            58
way          57
app.         56
go           56
android      56
used         55
best         54
download     53
better       53
play         52
.            51
give         50
send         49
people       48
able         48
message      47
since        47
many         46
works        45
apps         45
ads          45
keep         45
old          44
dtype: int64

In [135]:
def word_only_letters(word):
    for c in word:
        cat = unicodedata.category(c)
        if cat not in ('Ll','Lu'):  #only letter upper y lower
            return False
    return True

# clean only letter
def text_only_letters(text):
    if text is not None:
        #list of the word in the text
        words = text.strip().split()
        words_filtered = []
        for word in words:
            if word_only_letters(word):
                words_filtered.append(word)
            result = " ".join(words_filtered) #join the word in a text with space separation
    else:
        result = None
    return result

In [136]:
df_interim['review'] = df_interim['review'].apply(text_only_letters)

In [137]:
#remove multi letter looove iiiitttt, or repeat secuence
def replace_multiple_letters(message):
  if message is not None:
    result = re.sub(r'(.+?)\1+', r'\1', message)
    #result = re.sub(r"([a-zA-Z])＼1{2,}", r"＼1", message)
  else:
    result = None
  return result



In [138]:
df_interim['review'] = df_interim['review'].apply(replace_multiple_letters)


In [139]:
df_interim['review']


0      privacy least put option apear mean people lik...
1      mesenger isues ever since last update initial ...
2      profile time wife anybody one post view would ...
3      new features suck us working back buton guys m...
4      forced reload uploading pic replying coment la...
                             ...                        
886    loved incredible awesome go power make new cla...
887    time legendary game birthday party levels shor...
888    ads way heavy listen bad ads every round wheth...
889    fun works perfectly ads anoying think especial...
890    everywhere se angry birds everywhere stop play...
Name: review, Length: 891, dtype: object

In [140]:
#copy to df
df = df_interim.copy()


Setp 3 - Model

In [141]:
# Separate target and predictor
X = df['review']
y = df['polarity']

In [142]:
X=df['review']
y=df['polarity']

In [143]:
#Split your data in train and test set
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=42) #, test_size=0.25

In [144]:

#Vectorizer change matriz with 0-1
vec = CountVectorizer(stop_words = "english")
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()



In [145]:
X_train


array([[1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 1, 0]])

In [146]:
X_train.shape

(668, 2796)

In [147]:
vec.get_feature_names_out()

array(['ability', 'abke', 'able', ..., 'zespole', 'zom', 'zoming'],
      dtype=object)

In [148]:
#Naive Bayes
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB()

In [149]:
MultinomialNB()

MultinomialNB()

In [150]:
#score of train dataset 
nb.score(X_train, y_train)

0.9520958083832335

In [151]:
#score test dataset
nb.score(X_test, y_test)


0.820627802690583

In [152]:
nb.predict(vec.transform(['i don\'t like','eat much ram','eat','like']))

array([0, 0, 0, 0])

In [153]:
nb.predict(vec.transform(['love','great solid love important feature','ram','like fast']))

array([1, 1, 0, 1])

In [154]:
df[df['polarity']==1]['review'].sample(10)

665    hamro keyboard fantastic ap mobile write nepal...
676    god aps rlrl rzrlrl rr rsrzrlrl rzrl rlrzrz io...
173    new theme fun great se new items aded frozen s...
873    awesome game love realy fun games definitely s...
736    best mobile browser ever opera mini stil best ...
540    definitely worth many lousy whiners reviewing ...
847                                      god rsrz rzrlrl
525    great best cros platform cloud storage awesome...
491                        good ap great features overal
698    almost every time scrol webpage copy dialog ap...
Name: review, dtype: object

In [155]:
df[df['polarity']==0]['review'].sample(10)

498    tablet suport chance geting tablet suport anytime
57     much white space everything spaced also notifi...
711    slow weird lack realy slow sure god search bar...
787    geting used aficionado lately eat much ram pho...
217    certain update hapened recent update got rid t...
284    eroneous ap wechat ap loading location data th...
137    love playing candy crush u go another episode ...
503                     working great worse bugd without
231    update sucks friends play coc vilages sems for...
362    please fix voice video know nice kind sound me...
Name: review, dtype: object

* save the model to file

* pickle.dump(nb, open(filename, 'wb'))


* pickle.dump(nb, open("../models/modelo_final.sav", 'wb'))



In [156]:
#filename = 'modelo_final.sav' #use absolute path
pickle.dump(nb, open('../models/modelo_final.sav', 'wb'))